In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
!pip install transformers


In [6]:
!pip install torch

In [7]:
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer
import torch
import numpy as np

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [9]:
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
 
tokenizer_for_bert = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')


In [58]:
def bert_question_answer(question, passage, max_len=512):
    #Tokenize input question and passage 
    #Add special tokens - [CLS] and [SEP]
    input_ids = tokenizer_for_bert.encode (question, passage,  max_length= max_len, truncation=True)  

 
    #Getting number of tokens in 1st sentence (question) and 2nd sentence (passage that contains answer)
    sep_index = input_ids.index(102) 
    len_question = sep_index + 1  
    len_passage = len(input_ids)- len_question  

     
    #Need to separate question and passage
    #Segment ids will be 0 for question and 1 for passage
    segment_ids =  [0]*len_question + [1]*(len_passage)  
  
    #Converting token ids to tokens
    tokens = tokenizer_for_bert.convert_ids_to_tokens(input_ids) 
 
    #Getting start and end scores for answer
    #Converting input arrays to torch tensors before passing to the model
    start_token_scores = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([segment_ids]) )[0]
    end_token_scores = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([segment_ids]) )[1]
 
    #Converting scores tensors to numpy arrays
    start_token_scores = start_token_scores.detach().numpy().flatten()
    end_token_scores = end_token_scores.detach().numpy().flatten()

    #Getting start and end index of answer based on highest scores
    answer_start_index = np.argmax(start_token_scores)
    answer_end_index = np.argmax(end_token_scores)

 
    #Getting scores for start and end token of the answer
    start_token_score = np.round(start_token_scores[answer_start_index], 2)
    end_token_score = np.round(end_token_scores[answer_end_index], 2)

 
    #Combining subwords starting with ## and get full words in output. 
    #It is because tokenizer breaks words which are not in its vocab.
    answer = tokens[answer_start_index] 
    for i in range(answer_start_index + 1, answer_end_index + 1):
        if tokens[i][0:2] == '##':  
            answer += tokens[i][2:] 
        else:
            answer += ' ' + tokens[i]  
 
    # If the answer didn't find in the passage
    if ( answer_start_index == 0) or (start_token_score < 0 ) or  (answer == '[SEP]') or ( answer_end_index <  answer_start_index):
        answer = "Sorry!, I could not find an answer in the passage."
     
    return (answer_start_index, answer_end_index, start_token_score, end_token_score,  answer)


In [59]:
passage= """In pursuance of a resolution passed at a public meeting of the Hindus of the city of Colombo, we beg to offer you a hearty welcome to this Island. We deem it a privilege to be the first to welcome you on your return home from your great mission in the West.
We have watched with joy and thankfulness the success with which the mission has, under God's blessing, been crowned. You have proclaimed to the nations of Europe and America the Hindu ideal of a universal religion, harmonising all creeds, providing spiritual food for each soul according to its needs, and lovingly drawing it unto God. You have preached the Truth and the Way, taught from remote ages by a succession of Masters whose blessed feet have walked and sanctified the soil of India, and whose gracious presence and inspiration have made her, through all her vicissitudes, the Light of the World.
To the inspiration of such a Master, Shri Ramakrishna Paramahamsa Deva, and to your self-sacrificing zeal, Western nations owe the priceless boon of being placed in living contact with the spiritual genius of India, while to many of our own countrymen, delivered from the glamour of Western civilisation, the value of Our glorious heritage has been brought home.
By your noble work and example you have laid humanity under an obligation difficult to repay, and you have shed fresh lustre upon our Motherland. We pray that the grace of God may continue to prosper you and your work, and
We remain, Revered Sir,
Yours faithfully,
for and on behalf of the Hindus of Colombo,
P. COOMARA SWAMY,
Member of the Legislative Council of Ceylon,
Chairman of the Meeting.
A. KULAVEERASINGHAM, Secretary.
Colombo, January, 1897.
The Swami gave a brief reply, expressing his appreciation of the kind welcome he had received. He took advantage of the opportunity to point out that the demonstration had not been made in honour of a great politician, or a great soldier, or a millionaire, but of a begging Sannyâsin, showing the tendency of the Hindu mind towards religion. He urged the necessity of keeping religion as the backbone of the national life if the nation were to live, and disclaimed any personal character for the welcome he had received, but insisted upon its being the recognition of a principle.
On the evening of the 16th the Swami gave the following public lecture in the Floral Hall:
What little work has been done by me has not been from any inherent power that resides in me, but from the cheers, the goodwill, the blessings that have followed my path in the West from this our very beloved, most sacred, dear Motherland. Some good has been done, no doubt, in the West, but specially to myself; for what before was the result of an emotional nature, perhaps, has gained the certainty of conviction and attained the power and strength of demonstration. Formerly I thought as every Hindu thinks, and as the Hon. President has just pointed out to you, that this is the Punya Bhumi, the land of Karma. Today I stand here and say, with the conviction of truth, that it is so. If there is any land on this earth that can lay claim to be the blessed Punya Bhumi, to be the land to which all souls on this earth must come to account for Karma, the land to which every soul that is wending its way Godward must come to attain its last home, the land where humanity has attained its highest towards gentleness, towards generosity, towards purity, towards calmness, above all, the land of introspection and of spirituality — it is India. Hence have started the founders of religions from the most ancient times, deluging the earth again and again with the pure and perennial waters of spiritual truth. Hence have proceeded the tidal waves of philosophy that have covered the earth, East or West, North or South, and hence again must start the wave which is going to spiritualise the material civilisation of the world. Here is the life-giving water with which must be quenched the burning fire of materialism which is burning the core of the hearts of millions in other lands. Believe me, my friends, this is going to be.
So much I have seen, and so far those of you who are students of the history of races are already aware of this fact. The debt which the world owes to our Motherland is immense. Taking country with country, there is not one race on this earth to which the world owes so much as to the patient Hindu, the mild Hindu. "The mild Hindu" sometimes is used as an expression of reproach; but if ever a reproach concealed a wonderful truth, it is in the term, "the mild Hindu", who has always been the blessed child of God. Civilisations have arisen in other parts of the world. In ancient times and in modern times, great ideas have emanated from strong and great races. In ancient and in modern times, wonderful ideas have been carried forward from one race to another. In ancient and in modern times, seeds of great truth and power have been cast abroad by the advancing tides of national life; but mark you, my friends, it has been always with the blast of war trumpets and with the march of embattled cohorts. Each idea had to be soaked in a deluge of blood. Each idea had to wade through the blood of millions of our fellow-beings. Each word of power had to be followed by the groans of millions, by the wails of orphans, by the tears of widows. This, in the main, other nations have taught; but India has for thousands of years peacefully existed. Here activity prevailed when even Greece did not exist, when Rome was not thought of, when the very fathers of the modern Europeans lived in the forests and painted themselves blue. Even earlier, when history has no record, and tradition dares not peer into the gloom of that intense past, even from then until now, ideas after ideas have marched out from her, but every word has been spoken with a blessing behind it and peace before it. We, of all nations of the world, have never been a conquering race, and that blessing is on our head, and therefore we live.
There was a time when at the sound of the march of big Greek battalions the earth trembled. Vanished from off the face of the earth, with not every a tale left behind to tell, gone is that ancient land of the Greeks. There was a time when the Roman Eagle floated over everything worth having in this world; everywhere Rome's power was felt and pressed on the head of humanity; the earth trembled at the name of Rome. But the Capitoline Hill is a mass of ruins, the spider weaves its web where the Caesars ruled. There have been other nations equally glorious that have come and gone, living a few hours of exultant and exuberant dominance and of a wicked national life, and then vanishing like ripples on the face of the waters. Thus have these nations made their mark on the face of humanity. But we live, and if Manu came back today he would not be bewildered, and would not find himself in a foreign land. The same laws are here, laws adjusted and thought out through thousands and thousands of years; customs, the outcome of the acumen of ages and the experience of centuries, that seem to be eternal; and as the days go by, as blow after blow of misfortune has been delivered upon them, such blows seem to have served one purpose only, that of making them stronger and more constant. And to find the centre of all this, the heart from which the blood flows, the mainspring of the national life, believe me when I say after my experience of the world, that it is here.
To the other nations of the world, religion is one among the many occupations of life. There is politics, there are the enjoyments of social life, there is all that wealth can buy or power can bring, there is all that the senses can enjoy; and among all these various occupations of life and all this searching after something which can give yet a little more whetting to the cloyed senses — among all these, there is perhaps a little bit of religion. But here, in India, religion is the one and the only occupation of life. How many of you know that there has been a Sino-Japanese War? Very few of you, if any. That there are tremendous political movements and socialistic movements trying to transform Western society, how many of you know? Very few indeed, if any. But that there was a Parliament of Religions in America, and that there was a Hindu Sannyâsin sent over there, I am astonished to find that even the cooly knows of it. That shows the way the wind blows, where the national life is. I used to read books written by globe-trotting travellers, especially foreigners, who deplored the ignorance of the Eastern masses, but I found out that it was partly true and at the same time partly untrue. If you ask a ploughman in England, or America, or France, or Germany to what party he belongs, he can tell you whether he belongs to the Radicals or the Conservatives, and for whom he is going to vote. In America he will say whether he is Republican or Democrat, and he even knows something about the silver question. But if you ask him about his religion, he will tell you that he goes to church and belongs to a certain denomination. That is all he knows, and he thinks it is sufficient.
Now, when we come to India, if you ask one of our ploughmen, "Do you know anything about politics?" He will reply, "What is that?" He does not understand the socialistic movements, the relation between capital and labour, and all that; he has never heard of such things in his life, he works hard and earns his bread. But you ask, "What is your religion?" he replies, "Look here, my friend, I have marked it on my forehead." He can give you a good hint or two on questions of religion. That has been my experience. That is our nation's life.
Individuals have each their own peculiarities, and each man has his own method of growth, his own life marked out for him by the infinite past life, by all his past Karma as we Hindus say. Into this world he comes with all the past on him, the infinite past ushers the present, and the way in which we use the present is going to make the future. Thus everyone born into this world has a bent, a direction towards which he must go, through which he must live, and what is true of the individual is equally true of the race. Each race, similarly, has a peculiar bent, each race has a peculiar raison d'être, each race has a peculiar mission to fulfil in the life of the world. Each race has to make its own result, to fulfil its own mission. Political greatness or military power is never the mission of our race; it never was, and, mark my words, it never will be. But there has been the other mission given to us, which is to conserve, to preserve, to accumulate, as it were, into a dynamo, all the spiritual energy of the race, and that concentrated energy is to pour forth in a deluge on the world whenever circumstances are propitious. Let the Persian or the Greek, the Roman, the Arab, or the Englishman march his battalions, conquer the world, and link the different nations together, and the philosophy and spirituality of India is ever ready to flow along the new-made channels into the veins of the nations of the world. The Hindu's calm brain must pour out its own quota to give to the sum total of human progress. India's gift to the world is the light spiritual.
Thus, in the past, we read in history that whenever there arose a greet conquering nation uniting the different races of the world, binding India with the other races, taking her out, as it were, from her loneliness and from her aloofness from the rest of the world into which she again and again cast herself, that whenever such a state has been brought about, the result has been the flooding of the world with Indian spiritual ideas. At the beginning of this century, Schopenhauer, the great German philosopher, studying from a not very clear translation of the Vedas made from an old translation into Persian and thence by a young Frenchman into Latin, says, "In the whole world there is no study so beneficial and so elevating as that of the Upanishads. It has been the solace of my life, it will be the solace of my death." This great German sage foretold that "The world is about to see a revolution in thought more extensive and more powerful than that which was witnessed by the Renaissance of Greek Literature", and today his predictions are coming to pass. Those who keep their eyes open, those who understand the workings in the minds of different nations of the West, those who are thinkers and study the different nations, will find the immense change that has been produced in the tone, the procedure, in the methods, and in the literature of the world by this slow, never-ceasing permeation of Indian thought.
But there is another peculiarity, as I have already hinted to you. We never preached our thoughts with fire and sword. If there is one word in the English language to represent the gift of India to the world, if there is one word in the English language to express the effect which the literature of India produces upon mankind, it is this one word, "fascination". It is the opposite of anything that takes you suddenly; it throws on you, as it were, a charm imperceptibly. To many, Indian thought, Indian manners; Indian customs, Indian philosophy, Indian literature are repulsive at the first sight; but let them persevere, let them read, let them become familiar with the great principles underlying these ideas, and it is ninety-nine to one that the charm will come over them, and fascination will be the result. Slow and silent, as the gentle dew that falls in the morning, unseen and unheard yet producing a most tremendous result, has been the work of the calm, patient, all-suffering spiritual race upon the world of thought.
Once more history is going to repeat itself. For today, under the blasting light of modern science, when old and apparently strong and invulnerable beliefs have been shattered to their very foundations, when special claims laid to the allegiance of mankind by different sects have been all blown into atoms and have vanished into air, when the sledge-hammer blows of modern antiquarian researches are pulverising like masses of porcelain all sorts of antiquated orthodoxies, when religion in the West is only in the hands of the ignorant and the knowing ones look down with scorn upon anything belonging to religion, here comes to the fore the philosophy of India, which displays the highest religious aspirations of the Indian mind, where the grandest philosophical facts have been the practical spirituality of the people. This naturally is coming to the rescue, the idea of the oneness of all, the Infinite, the idea of the Impersonal, the wonderful idea of the eternal soul of man, of the unbroken continuity in the march of beings, and the infinity of the universe. The old sects looked upon the world as a little mud-puddle and thought that time began but the other day. It was there in our old books, and only there that the grand idea of the infinite range of time, space, and causation, and above all, the infinite glory of the spirit of man governed all the search for religion. When the modern tremendous theories of evolution and conservation of energy and so forth are dealing death blows to all sorts of crude theologies, what can hold any more the allegiance of cultured humanity but the most wonderful, convincing, broadening, and ennobling ideas that can be found only in that most marvellous product of the soul of man, the wonderful voice of God, the Vedanta?
At the same time, I must remark that what I mean by our religion working upon the nations outside of India comprises only the principles, the background, the foundation upon which that religion is built. The detailed workings, the minute points which have been worked out through centuries of social necessity, little ratiocinations about manners and customs and social well-being, do not rightly find a place in the category of religion. We know that in our books a clear distinction is made between two sets of truths. The one set is that which abides for ever, being built upon the nature of man, the nature of the soul, the soul's relation to God, the nature of God, perfection, and so on; there are also the principles of cosmology, of the infinitude of creation, or more correctly speaking — projection, the wonderful law of cyclical procession, and so on — these are the eternal principles founded upon the universal laws in nature. The other set comprises the minor laws which guided the working of our everyday life They belong more properly to the Purânas, to the Smritis, and not to the Shrutis. These have nothing to do with the other principles. Even in our own nation these minor laws have been changing all the time. Customs of one age, of one Yuga, have not been the customs of another, and as Yuga comes after Yuga, they will still have to change. Great Rishis will appear and lead us to customs and manners that are suited to new environments.
The great principles underlying all this wonderful, infinite, ennobling, expansive view of man and God and the world have been produced in India. In India alone man has not stood up to fight for a little tribal God, saying "My God is true and yours is not true; let us have a good fight over it." It was only here that such ideas did not occur as fighting for little gods. These great underlying principles, being based upon the eternal nature of man, are as potent today for working for the good of the human race as they were thousands of years ago, and they will remain so, so tong as this earth remains, so long as the law of Karma remains, so long as we are born as individuals and have to work out our own destiny by our individual power.
And above all, what India has to give to the world is this. If we watch the growth and development of religions in different races, we shall always find this that each tribe at the beginning has a god of its own. If the tribes are allied to each other, these gods will have a generic name, as for example, all the Babylonian gods had. When the Babylonians were divided into many races, they had the generic name of Baal, just as the Jewish races had different gods with the common name of Moloch; and at the same time you will find that one of these tribes becomes superior to the rest, and lays claim to its own king as the king over all. Therefrom it naturally follows that it also wants to preserve its own god as the god of all the races. Baal-Merodach, said the Babylonians, was the greatest god; all the others were inferior. Moloch-Yahveh was the superior over all other Molochs. And these questions had to be decided by the fortunes of battle. The same struggle was here also. In India the same competing gods had been struggling with each other for supremacy, but the great good fortune of this country and of the world was that there came out in the midst of the din and confusion a voice which declared  — "That which exists is One; sages call It by various names." It is not that Shiva is superior to Vishnu, not that Vishnu is everything and Shiva is nothing, but it is the same one whom you call either Shiva, or Vishnu, or by a hundred other names. The names are different, but it is the same one. The whole history of India you may read in these few words. The whole history has been a repetition in massive language, with tremendous power, of that one central doctrine. It was repeated in the land till it had entered into the blood of the nation, till it began to tingle with every drop of blood that flowed in its veins, till it became one with the life, part and parcel of the material of which it was composed; and thus the land was transmuted into the most wonderful land of toleration, giving the right to welcome the various religions as well as all sects into the old mother-country.
And herein is the explanation of the most remarkable phenomenon that is only witnessed here — all the various sects, apparently hopelessly contradictory, yet living in such harmony. You may be a dualist, and I may be a monist. You may believe that you are the eternal servant of God, and I may declare that I am one with God Himself; yet both of us are good Hindus. How is that possible? Read then  — "That which exists is One; sages call It by various names." Above all others, my countrymen, this is the one grand truth that we have to teach to the world. Even the most educated people of other countries turn up their noses at an angle of forty-five degrees and call our religion idolatry. I have seen that; and they never stopped to think what a mass of superstition there was in their own heads. It is still so everywhere, this tremendous sectarianism, the low narrowness of the mind. The thing which a man has is the only thing worth having; the only life worth living is his own little life of dollar-worship and mammon-worship; the only little possession worth having is his own property, and nothing else. If he can manufacture a little clay nonsense or invent a machine, that is to be admired beyond the greatest possessions. That is the case over the whole world in spite of education and learning. But education has yet to be in the world, and civilisation — civilisation has begun nowhere yet. Ninety-nine decimal nine per cent of the human race are more or less savages even now. We may read of these things in books, and we hear of toleration in religion and all that, but very little of it is there yet in the world; take my experience for that. Ninety-nine per cent do not even think of it. There is tremendous religious persecution yet in every country in which I have been, and the same old objections are raised against learning anything new. The little toleration that is in the world, the little sympathy that is yet in the world for religious thought, is practically here in the land of the Aryan, and nowhere else. It is here that Indians build temples for Mohammedans and Christians; nowhere else. If you go to other countries and ask Mohammedans or people of other religions to build a temple for you, see how they will help. They will instead try to break down your temple and you too if they can. The one great lesson, therefore, that the world wants most, that the world has yet to learn from India, is the idea not only of toleration, but of sympathy. Well has it been said in the Mahimnah-stotra: "As the different rivers, taking their start from different mountains, running straight or crooked, at last come unto the ocean, so, O Shiva, the different paths which men take through different tendencies, various though they appear, crooked or straight, all lead unto These." Though they may take various roads, all are on the ways. Some may run a little crooked, others may run straight, but at last they will all come unto the Lord, the One. Then and then alone, is your Bhakti of Shiva complete when you not only see Him in the Linga, but you see Him everywhere. He is the sage, he is the lover of Hari who sees Hari in everything and in everyone. If you are a real lover of Shiva, you must see Him in everything and in everyone. You must see that every worship is given unto Him whatever may be the name or the form; that all knees bending towards the Caaba, or kneeling in a Christian church, or in a Buddhist temple are kneeling to Him whether they know it or not, whether they are conscious of it or not; that in whatever name or form they are offered, all these flowers are laid at His feet; for He is the one Lord of all, the one Soul of all souls. He knows infinitely better what this world wants than you or I. It is impossible that all difference can cease; it must exist; without variation life must cease. It is this clash, the differentiation of thought that makes for light, for motion, for everything. Differentiation, infinitely contradictory, must remain, but it is not necessary that we should hate each other therefore; it is not necessary therefore that we should fight each other.
Therefore we have again to learn the one central truth that was preached only here in our Motherland, and that has to be preached once more from India. Why? Because not only is it in our books, but it runs through every phase of our national literature and is in the national life. Here and here alone is it practiced every day, and any man whose eyes are open can see that it is practiced here and here alone. Thus we have to teach religion. There are other and higher lessons that India can teach, but they are only for the learned. The lessons of mildness, gentleness, forbearance, toleration, sympathy, and brotherhood, everyone may learn, whether man, woman, or child, learned or unlearned, without respect of race, caste, or creed. "They call Thee by various names; Thou art One."""
 
print (f'Length of the passage: {len(passage.split())} words')
 
 
question ="Who wrote the letter welcoming the Swami to the island of Colombo?"
print ('\nQuestion 1:\n', question)
_, _ , _ , _, ans  = bert_question_answer( question, passage)
print('\nAnswer from BERT: ', ans ,  '\n')
 
question ="What was the Swami's response to the warm welcome he received?"
print ('\nQuestion 2:\n', question)
_, _ , _ , _, ans  = bert_question_answer( question, passage)
print('\nAnswer from BERT: ', ans ,  '\n')
 
question ="What is the mission of the Hindu race according to Swami Vivekananda?"
print ('\nQuestion 3:\n', question)
_, _ , _ , _, ans  = bert_question_answer( question, passage)
print('\nAnswer from BERT: ', ans ,  '\n')
 
question ="How did the Swami describe the debt the world owes to India?"
print ('\nQuestion 4:\n', question)
_, _ , _ , _, ans  = bert_question_answer( question, passage)
print('\nAnswer from BERT: ', ans ,  '\n')
 
question ="What is the unique mission of the Hindu race according to the Swami?"
print ('\nQuestion 5:\n', question)
_, _ , _ , _, ans  = bert_question_answer( question, passage)
print('\nAnswer from BERT: ', ans ,  '\n')


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Length of the passage: 4460 words

Question 1:
 Who wrote the letter welcoming the Swami to the island of Colombo?


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.



Answer from BERT:  p . coomara swamy , member of the legislative council of ceylon , chairman of the meeting . a . kulaveerasingham 


Question 2:
 What was the Swami's response to the warm welcome he received?

Answer from BERT:  brief reply 


Question 3:
 What is the mission of the Hindu race according to Swami Vivekananda?


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.



Answer from BERT:  universal religion 


Question 4:
 How did the Swami describe the debt the world owes to India?


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.



Answer from BERT:  difficult to repay 


Question 5:
 What is the unique mission of the Hindu race according to the Swami?

Answer from BERT:  Sorry!, I could not find an answer in the passage. 



In [60]:
#@title Question-Answering Application { vertical-output: true }
#@markdown ---
question= "what is Deep learning" #@param {type:"string"}
passage = """Deep learning is a subset of machine learning which is essentially a neural network with three or more layers. These neural networks attempt to simulate the behavior of the human brain—albeit far from matching its ability—allowing it to “learn” from large amounts of data. While a neural network with a single layer can still make approximate predictions, additional hidden layers can help to optimize and refine for accuracy.

Deep learning drives many artificial intelligence (AI) applications and services that improve automation, performing analytical and physical tasks without human intervention. Deep learning technology lies behind everyday products and services (such as digital assistants, voice-enabled TV remotes, and credit card fraud detection) as well as emerging technologies (such as self-driving cars)."""
 
_, _ , _ , _, ans  = bert_question_answer( question, passage)
 
#@markdown Answer:
print(ans)

a subset of machine learning
